# IMPORTS

In [1]:
##################
# IMPORT MODULES #
##################
# SYS IMPORT
import os, inspect, importlib, argparse
import random
import gc
import time
import pandas as pd
import numpy as np
from pathlib import Path

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# UTILS

In [2]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

seed_everything(95)

In [3]:
from sklearn.feature_selection import chi2, f_classif, f_regression
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold

# From https://github.com/abhishekkrthakur/approachingalmost
class UnivariateFeatureSelction:
    def __init__(self, n_features, problem_type, scoring, return_cols=True):
        """
        Custom univariate feature selection wrapper on
        different univariate feature selection models from
        scikit-learn.
        :param n_features: SelectPercentile if float else SelectKBest
        :param problem_type: classification or regression
        :param scoring: scoring function, string
        """
        self.n_features = n_features
        
        if problem_type == "classification":
            valid_scoring = {
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function")
            
        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features
            )
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile=int(n_features * 100)
            )
        else:
            raise Exception("Invalid type of feature")
    
    def fit(self, X, y):
        return self.selection.fit(X, y)
    
    def transform(self, X):
        return self.selection.transform(X)
    
    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)
    
    def return_cols(self, X):
        if isinstance(self.n_features, int):
            mask = SelectKBest.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
                    
        elif isinstance(self.n_features, float):
            mask = SelectPercentile.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
        else:
            raise Exception("Invalid type of feature")
        
        return selected_features

# CONFIG

In [4]:
config = {
    "DATA_PATH" : "D:/Documents/GitHub/ml-pipeline/data/TPS-AUG2021/train.csv",
    "TARGET_VAR" : "loss"
}

# DATA & FEATURE ENGINEERING

In [5]:
df = pd.read_csv(config["DATA_PATH"])
target = df[config["TARGET_VAR"]]
df.head()

In [ ]:
from sklearn import preprocessing

def standardize(dataframe):
    scaler = preprocessing.StandardScaler()
    features = dataframe.columns[1:101]
    dataframe[features] = scaler.fit_transform(dataframe[features])
    return dataframe

def feature_engineering(dataframe, train=False):
    dataframe = standardize(dataframe)
    features = dataframe.columns[1:101]
    return dataframe, features

In [ ]:
df, features = feature_engineering(df)
df.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-1.668045,0.179768,0.498854,-1.767452,-0.585085,1.548611,-0.731714,1.096083,-0.163171,...,-0.576275,0.287884,-0.636568,1.048811,-0.066279,-0.659995,1.706200,1.113531,0.432237,15
1,1,0.887510,2.208234,-0.431857,-0.732528,3.907814,-0.863906,-0.021877,0.556142,-0.129652,...,-0.592941,-0.988442,-0.360043,-0.977224,-0.390478,-1.790770,-0.433968,1.218930,-1.222692,3
2,2,-0.628150,-0.763705,-0.408204,-0.546028,-0.364251,2.101452,-0.727663,-1.095000,0.178041,...,-0.570408,0.985834,1.358390,0.439352,-0.101587,-0.224966,0.917008,0.185501,0.753750,6
3,3,-0.975889,-0.810879,-0.547278,1.131234,-0.611685,-0.753071,0.113171,1.238672,-0.643465,...,-0.570146,-1.478323,-0.595755,0.744873,-1.272692,0.235002,0.352117,-0.186368,0.350606,2
4,4,-0.232366,-0.740119,0.651167,-0.179911,-0.579810,0.748861,-0.503574,-0.749140,0.077481,...,-0.537094,-0.469704,-0.327684,-0.946548,-0.017820,-0.248517,1.277657,0.321752,-0.172972,1


# TRAINING A BASELINE WITH ALL FEATURES

In [ ]:
metric = mean_squared_error
train_x, valid_x, train_y, valid_y = train_test_split(df[features], target, test_size=0.2, random_state=95)

In [ ]:
XGB_REGRESSION = {
    "objective": "reg:squarederror",
    "n_estimators" : 10000,
    "max_depth": 10,
    "learning_rate": 0.006,
    "colsample_bytree": 0.5,
    "subsample": 0.6,
    "reg_alpha" : 0.006221417528979453,
    "reg_lambda": 3.178956727410822e-07,
    "min_child_weight": 123,
    "n_jobs": 2,
    "seed": 95,
    'tree_method': "gpu_hist",
    "gpu_id": 0,
    'predictor': 'gpu_predictor'
}

model = xgb.XGBRegressor(**XGB_REGRESSION)

In [ ]:
model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], early_stopping_rounds=200, verbose=False)
preds = model.predict(valid_x)
score = metric(valid_y, preds, squared=False)
print(f"VALIDATION SCORE : {score}")

# UNIVARIATE FEATURE SELECTION

In [52]:
ufs = UnivariateFeatureSelction(
        n_features=0.8,
        problem_type="regression",
        scoring="f_regression"
)
columns = df[features].columns.values.tolist()
ufs.fit(df[columns], df[config["TARGET_VAR"]].values.ravel())
selected_features = ufs.return_cols(df[columns])

In [53]:
features

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20',
       'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30',
       'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40',
       'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50',
       'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60',
       'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70',
       'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80',
       'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90',
       'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99'],
      dtype='object')

In [54]:
selected_features

['f0',
 'f1',
 'f2',
 'f3',
 'f4',
 'f5',
 'f6',
 'f8',
 'f9',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f21',
 'f22',
 'f23',
 'f25',
 'f26',
 'f27',
 'f28',
 'f29',
 'f30',
 'f31',
 'f32',
 'f33',
 'f34',
 'f35',
 'f36',
 'f37',
 'f38',
 'f39',
 'f40',
 'f41',
 'f42',
 'f43',
 'f44',
 'f45',
 'f46',
 'f47',
 'f48',
 'f49',
 'f50',
 'f51',
 'f52',
 'f53',
 'f54',
 'f55',
 'f56',
 'f57',
 'f58',
 'f59',
 'f60',
 'f61',
 'f62',
 'f63',
 'f64',
 'f65',
 'f66',
 'f68',
 'f69',
 'f70',
 'f72',
 'f73',
 'f74',
 'f75',
 'f76',
 'f77',
 'f78',
 'f79',
 'f81',
 'f82',
 'f84',
 'f85',
 'f88',
 'f90',
 'f91',
 'f92',
 'f93',
 'f94',
 'f95',
 'f96',
 'f97',
 'f98',
 'f99']

# TRAIN A MODEL WITH SELECTED FEATURES

In [42]:
train_x_selected, valid_x_selected, train_y_selected, valid_y_selected = train_test_split(df[selected_features], target, test_size=0.2, random_state=95)

In [43]:
model_selected = xgb.XGBRegressor(**XGB_REGRESSION)

In [44]:
model_selected.fit(train_x_selected, train_y_selected, eval_set=[(valid_x_selected , valid_y_selected)], early_stopping_rounds=200, verbose=False)
preds_selected = model_selected.predict(valid_x_selected)
score_selected = metric(valid_y_selected, preds_selected, squared=False)
print(f"VALIDATION SCORE : {score_selected}")

VALIDATION SCORE : 7.795086006255788


# FEATURE SELECTION USING RECURSIVE FEATURE ELIMINATION (RFE)

In [55]:
from sklearn.feature_selection import RFE
rfe = RFE(model_selected, n_features_to_select = 0.8)
rfe.fit(train_x, train_y)
preds_selected_rfe = rfe.predict(valid_x)
score_selected_rfe = metric(valid_y, preds_selected_rfe, squared=False)
print(f"VALIDATION SCORE : {score_selected_rfe}")

In [ ]:
from operator import itemgetter
features = X_train.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    print(x, y)

# FEATURE SELECTION USING THE BORUTA SHAP METHOD

In [9]:
from BorutaShap import BorutaShap
selector = BorutaShap(model=model_selected, importance_measure="shap", classification=False)
selector.fit(train_x, train_y, n_trials=50, random_state=95)
selector.plot(which_features='all', figsize=(16,12))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\chopp\anaconda3\envs\ML-37\lib\site-packages\IPython\core\interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e78fa15da944>", line 2, in <module>
    selector = BorutaShap(model=model_selected, importance_measure="shap", classification=False)
NameError: name 'model_selected' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\chopp\anaconda3\envs\ML-37\lib\site-packages\IPython\core\interactiveshell.py", line 2054, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\chopp\anaconda3\envs\ML-37\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinner